<a href="https://colab.research.google.com/github/shakshi12/RumourGNN/blob/master/charlie_GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CharlieHebdo - Word Embeddings through GloVe and Rumour Detection

## Load csv file into pandas dataframe

---



In [4]:
from google.colab import files
uploaded = files.upload()

Saving dump_charliehebdo.csv to dump_charliehebdo (1).csv


In [1]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['dump_charliehebdo.csv']), header = None)

NameError: ignored

In [6]:

import pandas as pd
df = pd.read_csv('dump_charliehebdo.csv', header = None)

FileNotFoundError: ignored

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,552784600502915072,Charlie Hebdo became well known for publishing...,Wed Jan 07 11:11:33 +0000 2015,NaN,41591,552785249420447745,"Now 10 dead in a shooting there today RT ""@BBC...",Wed Jan 07 11:14:08 +0000 2015,Chicago,4671,0
1,552784600502915072,Charlie Hebdo became well known for publishing...,Wed Jan 07 11:11:33 +0000 2015,NaN,41591,552786761534144512,@BBCDanielS @BBCWorld I'm guessing this is bei...,Wed Jan 07 11:20:08 +0000 2015,South Carolina,59,0
2,552784600502915072,Charlie Hebdo became well known for publishing...,Wed Jan 07 11:11:33 +0000 2015,NaN,41591,552786803884060672,@BBCDanielS @BBCWorld why would you mention th...,Wed Jan 07 11:20:18 +0000 2015,NaN,96357,0
3,552784600502915072,Charlie Hebdo became well known for publishing...,Wed Jan 07 11:11:33 +0000 2015,NaN,41591,552786954656710656,@BBCDanielS @BBCWorld perps identified?,Wed Jan 07 11:20:54 +0000 2015,NaN,751,0
4,552784600502915072,Charlie Hebdo became well known for publishing...,Wed Jan 07 11:11:33 +0000 2015,NaN,41591,552787979224092672,@BBCDanielS @BBCWorld who is charlie hebdo?,Wed Jan 07 11:24:59 +0000 2015,Singapore,115,0


In [0]:
df = df.loc[:,[0,1,5,6,10]]

In [5]:
df.head()

,0,1,5,6,10
0,552784600502915072,Charlie Hebdo became well known for publishing...,552785249420447745,"Now 10 dead in a shooting there today RT ""@BBC...",0
1,552784600502915072,Charlie Hebdo became well known for publishing...,552786761534144512,@BBCDanielS @BBCWorld I'm guessing this is bei...,0
2,552784600502915072,Charlie Hebdo became well known for publishing...,552786803884060672,@BBCDanielS @BBCWorld why would you mention th...,0
3,552784600502915072,Charlie Hebdo became well known for publishing...,552786954656710656,@BBCDanielS @BBCWorld perps identified?,0
4,552784600502915072,Charlie Hebdo became well known for publishing...,552787979224092672,@BBCDanielS @BBCWorld who is charlie hebdo?,0


In [0]:
df.columns = ['src_id','src_text','reply_id','reply_text','labels']

In [7]:
df.head()

,src_id,src_text,reply_id,reply_text,labels
0,552784600502915072,Charlie Hebdo became well known for publishing...,552785249420447745,"Now 10 dead in a shooting there today RT ""@BBC...",0
1,552784600502915072,Charlie Hebdo became well known for publishing...,552786761534144512,@BBCDanielS @BBCWorld I'm guessing this is bei...,0
2,552784600502915072,Charlie Hebdo became well known for publishing...,552786803884060672,@BBCDanielS @BBCWorld why would you mention th...,0
3,552784600502915072,Charlie Hebdo became well known for publishing...,552786954656710656,@BBCDanielS @BBCWorld perps identified?,0
4,552784600502915072,Charlie Hebdo became well known for publishing...,552787979224092672,@BBCDanielS @BBCWorld who is charlie hebdo?,0


In [11]:
df.shape

(36189, 5)

In [0]:
import nltk

In [50]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
text = list()
src_line = df['src_text'].values.tolist()
for l in src_line:
    token = word_tokenize(l)
    token = [t.lower() for t in token]
    punct = str.maketrans('','',string.punctuation)
    translate = [t.translate(punct) for t in token]
    words = [word for word in translate if word.isalpha()]
    stop = set(stopwords.words('english'))
    words = [t for t in words if not t in stop]
    text.append(words)

In [0]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
r_text = list()
reply_line = df['reply_text'].values.tolist()
for l in reply_line:
    token = word_tokenize(l)
    token = [t.lower() for t in token]
    punct = str.maketrans('','',string.punctuation)
    translate = [t.translate(punct) for t in token]
    words = [word for word in translate if word.isalpha()]
    stop = set(stopwords.words('english'))
    words = [t for t in words if not t in stop]
    r_text.append(words)

In [94]:
len(text)

36189

In [27]:
len(r_text)

36189

In [0]:
import gensim

In [0]:
model = gensim.models.Word2Vec(sentences = text, size = 100, window = 2, workers = 3, min_count = 1)

In [96]:
words = list(model.wv.vocab)
print('Vocab size', len(words))

Vocab size 4032


In [0]:
file = 'word2vec_src.txt'

In [98]:
model.wv.save_word2vec_format(file,binary = False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
!cp word2vec_src.txt "drive/My Drive/"

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import numpy as np
emb = {}
f = open(os.path.join('', 'word2vec_src.txt'), encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    value = np.asarray(values[1:])
    emb[word] = value
f.close()

In [107]:
embed_mat[112]

array([ 2.2298550e-01, -2.0017240e-01,  9.1334140e-01,  3.4326557e-02,
       -6.2022343e-02,  1.9741693e-01, -1.3738480e-01, -9.1588396e-01,
        8.6108583e-01,  6.2921780e-01, -6.8700093e-01, -1.1293784e+00,
       -4.7708908e-01, -1.0140340e+00,  8.9787006e-01, -5.0860420e-01,
       -7.6187430e-01, -1.3420318e-01, -1.0711940e+00,  3.1554362e-01,
        3.4133235e-01,  1.0564967e+00,  9.9743130e-01,  3.7437510e-02,
        1.4061078e-01,  5.6215310e-01, -2.5689480e-01, -4.3761150e-02,
        4.2130467e-02,  3.3651060e-02, -1.4015785e+00,  8.7205386e-01,
        3.2287115e-01, -6.9617057e-01, -7.8128284e-01,  1.4838319e+00,
        4.9952410e-01, -1.0278416e+00,  9.5360667e-01, -7.8388860e-01,
        3.5543920e-01, -1.2351236e+00,  3.9143330e-01,  2.1897101e-01,
        6.5705700e-01, -2.8894280e-01,  8.3089060e-01,  1.1824386e+00,
        8.2176550e-01,  9.3494190e-01, -2.3361814e-01, -8.6627316e-01,
       -2.6684730e-01,  2.9594070e-01, -1.8774975e-02,  6.3586410e-01,
      

In [60]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


In [0]:
max_len = max([len(tweet.split()) for tweet in df.src_text])

In [108]:
token = Tokenizer()
token.fit_on_texts(text)
seq = token.texts_to_sequences(text)
index = token.word_index
print('unique no of tokens ', index)
padding = pad_sequences(seq, maxlen = max_len)
rumour = df['labels'].values
print('Shape of tweet ',padding.shape)
print('Shape of rumour ', rumour.shape)

unique no of tokens  {'http': 1, 'charliehebdo': 2, 'paris': 3, 'attack': 4, 'charlie': 5, 'hebdo': 6, 'jesuischarlie': 7, 'french': 8, 'police': 9, 'france': 10, 'people': 11, 'killed': 12, 'nt': 13, 'muslim': 14, 'muslims': 15, 'shooting': 16, 'cartoons': 17, 'islam': 18, 'speech': 19, 'amp': 20, 'suspects': 21, 'freedom': 22, 'cartoon': 23, 'breaking': 24, 'today': 25, 'say': 26, 'hostage': 27, 'gunmen': 28, 'attacks': 29, 'condemn': 30, 'says': 31, 'dead': 32, 'terrorist': 33, 'terrorists': 34, 'victims': 35, 'hostages': 36, 'free': 37, 'magazine': 38, 'media': 39, 'cartoonists': 40, 'reports': 41, 'world': 42, 'satirical': 43, 'kosher': 44, 'us': 45, 'one': 46, 'stand': 47, 'religion': 48, 'terror': 49, 'afp': 50, 'massacre': 51, 'two': 52, 'right': 53, 'support': 54, 'journalists': 55, 'live': 56, 'solidarity': 57, 'must': 58, 'like': 59, 'name': 60, 'much': 61, 'know': 62, 'expression': 63, 'cartoonist': 64, 'via': 65, 'killing': 66, 'islamic': 67, 'offices': 68, 'newspaper': 69

In [109]:
n = len(index) + 1
embed_mat = np.zeros((n, 100))
for s,values in index.items():
    print(s,values)
    if values>n:
        continue
    embed_vec = emb.get(s)
    if embed_vec is not None:
        embed_mat[values] = embed_vec
print(n)

http 1
charliehebdo 2
paris 3
attack 4
charlie 5
hebdo 6
jesuischarlie 7
french 8
police 9
france 10
people 11
killed 12
nt 13
muslim 14
muslims 15
shooting 16
cartoons 17
islam 18
speech 19
amp 20
suspects 21
freedom 22
cartoon 23
breaking 24
today 25
say 26
hostage 27
gunmen 28
attacks 29
condemn 30
says 31
dead 32
terrorist 33
terrorists 34
victims 35
hostages 36
free 37
magazine 38
media 39
cartoonists 40
reports 41
world 42
satirical 43
kosher 44
us 45
one 46
stand 47
religion 48
terror 49
afp 50
massacre 51
two 52
right 53
support 54
journalists 55
live 56
solidarity 57
must 58
like 59
name 60
much 61
know 62
expression 63
cartoonist 64
via 65
killing 66
islamic 67
offices 68
newspaper 69
parisshooting 70
least 71
prophet 72
ahmed 73
said 74
gunman 75
publish 76
merabet 77
supermarket 78
many 79
want 80
let 81
innocent 82
photo 83
grocery 84
kill 85
offensive 86
taken 87
satire 88
rip 89
scene 90
jews 91
rt 92
ever 93
silence 94
muhammad 95
strongly 96
cover 97
die 98
charb 99
pr

In [110]:
sorted_words = sorted(index)
print(sorted_words)

['aamir', 'abc', 'abdulela', 'able', 'abolish', 'abominable', 'aboujahjah', 'absolute', 'absolutely', 'absurdity', 'absurdly', 'abu', 'abukhadeejahsp', 'abuses', 'accept', 'accident', 'accomplice', 'according', 'account', 'acct', 'accused', 'acknowledge', 'across', 'act', 'action', 'actions', 'activists', 'acts', 'actual', 'actually', 'addaraqutni', 'added', 'admin', 'admitting', 'advanced', 'advice', 'advocate', 'advocates', 'affair', 'affected', 'affiliate', 'afp', 'afraid', 'africa', 'aftermath', 'age', 'agence', 'agencies', 'agenda', 'agmahdi', 'ago', 'agree', 'ahead', 'ahmadiyya', 'ahmed', 'ahmedmerabet', 'ai', 'aiming', 'air', 'airforce', 'airport', 'aisne', 'aixlesbains', 'ajayendar', 'ajeopinion', 'akbar', 'akber', 'akhbar', 'akp', 'al', 'alarm', 'albaghdadi', 'albert', 'aleppo', 'alert', 'algerian', 'ali', 'alikhbariyah', 'alive', 'aljazeera', 'allah', 'allahou', 'allahu', 'alleged', 'allegedly', 'allegiance', 'allies', 'allow', 'allowing', 'ally', 'almasryalyoum', 'almost', '

In [111]:
embed_mat['chat']

IndexError: ignored

In [0]:
features = {}

In [68]:
feat_df = pd.DataFrame(columns = sorted_words)
print(feat_df)

Empty DataFrame
Columns: [aamir, abc, abdulela, able, abolish, abominable, aboujahjah, absolute, absolutely, absurdity, absurdly, abu, abukhadeejahsp, abuses, accept, accident, accomplice, according, account, acct, accused, acknowledge, across, act, action, actions, activists, acts, actual, actually, addaraqutni, added, admin, admitting, advanced, advice, advocate, advocates, affair, affected, affiliate, afp, afraid, africa, aftermath, age, agence, agencies, agenda, agmahdi, ago, agree, ahead, ahmadiyya, ahmed, ahmedmerabet, ai, aiming, air, airforce, airport, aisne, aixlesbains, ajayendar, ajeopinion, akbar, akber, akhbar, akp, al, alarm, albaghdadi, albert, aleppo, alert, algerian, ali, alikhbariyah, alive, aljazeera, allah, allahou, allahu, alleged, allegedly, allegiance, allies, allow, allowing, ally, almasryalyoum, almost, aloft, along, alqaeda, alqaida, already, also, although, alu, ...]
Index: []

[0 rows x 4032 columns]


In [119]:
count = 0
features_matrix = pd.DataFrame()
for number in padding[count]:
  if count <= 36189:  
    print(count)  
    feat_df = pd.DataFrame(columns = sorted_words)
    count = count + 1
    for key,value in index.items():
        feat_df[key] = embed_mat[number]
        feature_1 = feat_df.sum(axis=0)
        features_matrix[number] = feature_1
        #print(features_matrix)

0
1
2


KeyboardInterrupt: ignored

In [118]:
features_matrix

,5,6,356,431,558,240,563,17,52,469,732
aamir,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abdulela,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
able,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abolish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
zefrog,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zeid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zuckerberg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
feat_df=feat_df.fillna(0)

In [71]:
feat_df.head()

,aamir,abc,abdulela,able,abolish,abominable,aboujahjah,absolute,absolutely,absurdity,absurdly,abu,abukhadeejahsp,abuses,accept,accident,accomplice,according,account,acct,accused,acknowledge,across,act,action,actions,activists,acts,actual,actually,addaraqutni,added,admin,admitting,advanced,advice,advocate,advocates,affair,affected,...,wour,wow,wrapup,wrenchingly,writer,writers,writes,writing,wrong,wrote,ws,wsomeone,wterrorism,wthe,wwf,xenophobia,ya,yale,yard,yday,year,years,yelling,yemen,yes,yesterday,yet,yo,york,yorker,young,youngblkrepub,youth,youtube,yr,zefrog,zeid,zone,zuckerberg,ﷺ
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.148056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.025224,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.473406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.111171,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.359449,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
feat_df2 = pd.DataFrame(columns = sorted_words)
for number in padding[1]:
    for key,value in index.items():
        if number == value:
            feat_df2[key] = embed_mat[number]

In [73]:
feat_df2=feat_df2.fillna(0)
feat_df2.head()

,aamir,abc,abdulela,able,abolish,abominable,aboujahjah,absolute,absolutely,absurdity,absurdly,abu,abukhadeejahsp,abuses,accept,accident,accomplice,according,account,acct,accused,acknowledge,across,act,action,actions,activists,acts,actual,actually,addaraqutni,added,admin,admitting,advanced,advice,advocate,advocates,affair,affected,...,wour,wow,wrapup,wrenchingly,writer,writers,writes,writing,wrong,wrote,ws,wsomeone,wterrorism,wthe,wwf,xenophobia,ya,yale,yard,yday,year,years,yelling,yemen,yes,yesterday,yet,yo,york,yorker,young,youngblkrepub,youth,youtube,yr,zefrog,zeid,zone,zuckerberg,ﷺ
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.148056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.025224,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.473406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.111171,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.359449,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
features_matrix = pd.DataFrame()
# Sum of 100 dimensions of 1 word
feature_1 = feat_df.sum(axis=0)
feature_2 = feat_df2.sum(axis = 0)
temp = [feature_1, feature_2]
features_matrix = pd.concat(temp, axis = 1)
features_matrix2 = pd.DataFrame(data = features_matrix)
features_matrix2 = features_matrix2.transpose()
features_matrix2

,aamir,abc,abdulela,able,abolish,abominable,aboujahjah,absolute,absolutely,absurdity,absurdly,abu,abukhadeejahsp,abuses,accept,accident,accomplice,according,account,acct,accused,acknowledge,across,act,action,actions,activists,acts,actual,actually,addaraqutni,added,admin,admitting,advanced,advice,advocate,advocates,affair,affected,...,wour,wow,wrapup,wrenchingly,writer,writers,writes,writing,wrong,wrote,ws,wsomeone,wterrorism,wthe,wwf,xenophobia,ya,yale,yard,yday,year,years,yelling,yemen,yes,yesterday,yet,yo,york,yorker,young,youngblkrepub,youth,youtube,yr,zefrog,zeid,zone,zuckerberg,ﷺ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.233887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.233887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from glove import Corpus, Glove

In [121]:
pip install glove_python

In [0]:
corpus = Corpus() 

In [2]:
df.head()

NameError: ignored

In [0]:
import re
import numpy as np
# function for cleaning data
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

In [0]:
df['clean'] = np.vectorize(remove_pattern)(df['src_text'], "@[\w]*")

In [126]:
df.head()

,src_id,src_text,reply_id,reply_text,labels,clean
0,552784600502915072,Charlie Hebdo became well known for publishing...,552785249420447745,"Now 10 dead in a shooting there today RT ""@BBC...",0,Charlie Hebdo became well known for publishing...
1,552784600502915072,Charlie Hebdo became well known for publishing...,552786761534144512,@BBCDanielS @BBCWorld I'm guessing this is bei...,0,Charlie Hebdo became well known for publishing...
2,552784600502915072,Charlie Hebdo became well known for publishing...,552786803884060672,@BBCDanielS @BBCWorld why would you mention th...,0,Charlie Hebdo became well known for publishing...
3,552784600502915072,Charlie Hebdo became well known for publishing...,552786954656710656,@BBCDanielS @BBCWorld perps identified?,0,Charlie Hebdo became well known for publishing...
4,552784600502915072,Charlie Hebdo became well known for publishing...,552787979224092672,@BBCDanielS @BBCWorld who is charlie hebdo?,0,Charlie Hebdo became well known for publishing...


In [0]:
df['clean_r'] = np.vectorize(remove_pattern)(df['reply_text'], "@[\w]*")

In [0]:
df['clean'] = df['clean'].str.replace("[^a-zA-Z#]", " ")

In [0]:
df['clean'] = df.clean.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

In [0]:
df.clean = df.clean.apply(lambda x: x.split())

In [0]:
from nltk.stem.porter import * 
stemmer = PorterStemmer() 
tokenized_tweet = df.clean.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [0]:
tokens = list(tokenized_tweet)

In [3]:
len(tokens)

NameError: ignored

In [0]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])    
df['clean'] = tokenized_tweet

In [0]:
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(tokens, window=10)

In [136]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [137]:
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')
keys = glove.dictionary.keys()
glove.dictionary.values()


dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [0]:
features['charlie'] = 0

In [139]:
features = pd.DataFrame(0, index = np.arange(100), columns = keys)
features.shape

(100, 4500)

In [0]:
for key in keys:
  features[key] = (glove.word_vectors[glove.dictionary[key]])

In [0]:
features_sum = features.sum(axis = 0)

In [0]:
features_final = pd.DataFrame(0, index = np.arange(100), columns = keys)

In [143]:
sentences = 0
counter = 0
for sentence in tokens:    
  for word in sentence:
    for column in features_final.columns:
      if word == column:
        features_final.loc[counter, word] = features_sum[word]
  counter = counter + 1
  print("counter ", counter)

counter  1
counter  2
counter  3
counter  4
counter  5
counter  6
counter  7
counter  8
counter  9
counter  10
counter  11
counter  12
counter  13
counter  14
counter  15
counter  16
counter  17
counter  18
counter  19
counter  20
counter  21
counter  22
counter  23
counter  24
counter  25
counter  26
counter  27
counter  28
counter  29
counter  30
counter  31
counter  32
counter  33
counter  34
counter  35
counter  36
counter  37
counter  38
counter  39
counter  40
counter  41
counter  42
counter  43
counter  44
counter  45
counter  46
counter  47
counter  48
counter  49
counter  50
counter  51
counter  52
counter  53
counter  54
counter  55
counter  56
counter  57
counter  58
counter  59
counter  60
counter  61
counter  62
counter  63
counter  64
counter  65
counter  66
counter  67
counter  68
counter  69
counter  70
counter  71
counter  72
counter  73
counter  74
counter  75
counter  76
counter  77
counter  78
counter  79
counter  80
counter  81
counter  82
counter  83
counter  84
c

In [0]:
features_final = features_final.fillna(0)

In [200]:
for i in range(36190, 38192):
  features_final[i,:] = 0.0

TypeError: ignored

In [199]:
features_final.shape

(36189, 4500)

In [0]:
features_final.to_csv('X.csv')

In [0]:
!cp X.csv "drive/My Drive/"

In [0]:
!cp X.csv "drive/My Drive/"


In [5]:
df.to_csv('drive/My Drive/charliehebdo.csv')

NameError: ignored

In [0]:
y = df['labels']

In [158]:
y.to_csv('y.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [0]:
import pandas as pd
X = pd.read_csv('drive/My Drive/X.csv')

In [0]:
from scipy import sparse
X_csr = sparse.csr_matrix(X)

In [0]:
y = pd.read_csv('y.csv')

In [0]:
import numpy as np
A = np.load('adjacencymatrix.npz')

In [10]:
type(A)

numpy.lib.npyio.NpzFile

In [0]:
!cp y.csv "drive/My Drive/"

## Graph

In [0]:
import networkx as nx

In [0]:
short_graph = df

In [0]:
G = nx.from_pandas_edgelist(df=short_graph, source='src_id', target='reply_id')

In [0]:
nodes = list(short_graph.src_id)
nodes1 = list(short_graph.reply_id)
nodes.extend(nodes1)
#print(nodes)

In [0]:
G.add_nodes_from(nodes_for_adding=nodes)

In [0]:
A = nx.adjacency_matrix(G)

In [17]:
type(A)

scipy.sparse.csr.csr_matrix

In [0]:
from scipy import sparse

sparse.save_npz("adjacencymatrix.npz", A)

In [18]:
pwd


'/content'

**tensorflow**

In [1]:
!sudo apt install graphviz libgraphviz-dev libcgraph6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
libcgraph6 is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [2]:
!pip install spektral

In [3]:
 !git clone https://github.com/danielegrattarola/spektral.git

fatal: destination path 'spektral' already exists and is not an empty directory.


In [4]:
cd spektral

/content/spektral


In [36]:
!git checkout tf2

Already on 'tf2'
Your branch is up to date with 'origin/tf2'.


In [20]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating spektral.egg-info
writing spektral.egg-info/PKG-INFO
writing dependency_links to spektral.egg-info/dependency_links.txt
writing requirements to spektral.egg-info/requires.txt
writing top-level names to spektral.egg-info/top_level.txt
writing manifest file 'spektral.egg-info/SOURCES.txt'
writing manifest file 'spektral.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/tests
copying tests/test_datasets.py -> build/lib/tests
copying tests/__init__.py -> build/lib/tests
creating build/lib/spektral
copying spektral/brain.py -> build/lib/spektral
copying spektral/chem.py -> build/lib/spektral
copying spektral/__init__.py -> build/lib/spektral
creating build/lib/tests/test_layers
copying tests/test_layers/test_ops.py -> build/lib/tests/test_layers
copying tests/test_layers/test_global_pooling.py -> build/lib/tests/tes

In [21]:
!pip install grpcio --upgrade

     |████████████████████████████████| 2.4MB 2.9MB/s 
  Found existing installation: grpcio 1.15.0
    Uninstalling grpcio-1.15.0:
      Successfully uninstalled grpcio-1.15.0


In [0]:
from spektral.datasets import citation
#data = citation.load_data('cora')
#A, X, y, train_mask, val_mask, test_mask = data

X = X_csr
l = []
import numpy as np
for i in range(360):
  l.append(True)
for i in range (35829):
  l.append(False)
train_mask = np.array(l)
l1 = []
for i in range(360):
  l1.append(False)
for i in range(829):
  l1.append(True)
for i in range(35000):
  l1.append(False)
val_mask = np.array(l1)
l2 = []
for i in range(31000):
  l2.append(False)
for i in range(5189):
  l2.append(True)
test_mask = np.array(l2)
y = np.zeros((36189,2))
n1 = df['labels']
n = list(n1)
for i in range(36189):
  if (n[i] == 1):
    y[i,1] = 1
  elif (n[i] == 0):
    y[i,0] = 1



N = A.shape[0]
F = X.shape[-1]
n_classes = y.shape[-1]

In [2]:
from spektral.datasets import citation
data = citation.load_data('cora')
A, X, y, train_mask, val_mask, test_mask = data
N = A.shape[0]
F = X.shape[-1]
n_classes = y.shape[-1]

Loading cora dataset
Pre-processing node features


In [39]:
A.shape

(2708, 2708)

In [23]:
N

38191

In [24]:
X.shape

(36189, 4501)

In [25]:
test_mask.shape
F

4501

In [3]:
from spektral.layers import GraphConv
from keras.models import Model
from keras.layers import Input, Dropout

Using TensorFlow backend.


In [4]:
# Model definition
X_in = Input(shape=(F, ))  # Input layer for X
A_in = Input((36189, ), sparse=True)  # Input layer for A

graph_conv_1 = GraphConv(16, activation='relu')([X_in, A_in])
dropout = Dropout(0.5)(graph_conv_1)
graph_conv_2 = GraphConv(n_classes, activation='softmax')([dropout, A_in])

# Build model
model = Model(inputs=[X_in, A_in], outputs=graph_conv_2)




Instructions for updating:
If using Keras pass *_constraint arguments to layers.


AttributeError: ignored

In [0]:
from keras.layers import Layer, LeakyReLU, Dropout


In [28]:
import keras
print(keras.__version__)

2.2.5


In [30]:
!pip uninstall keras

Uninstalling Keras-2.2.5:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.5


In [31]:
pip install keras==2.1.2

     |████████████████████████████████| 307kB 2.8MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.2 which is incompatible.


In [6]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 512kB 40.5MB/s 
     |████████████████████████████████| 3.8MB 47.1MB/s 
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0
